# MLB.com stats crawler
https://github.com/whyhugo/Data-Analysis-in-Sports
> Please input the URL which can get from https://www.mlb.com/stats

In [1]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from lxml import etree
import time

In [3]:

chrome_options = Options()
chrome_options.add_argument('--headless')



url = input('URL:')
driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe', options=chrome_options)
html = driver.get(url)
sp = soup(driver.page_source, 'html.parser')

URL: https://www.mlb.com/stats/arizona-diamondbacks?playerPool=ALL


C:\Users\user\AppData\Local\Temp\ipykernel_19720\4031441934.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe', options=chrome_options)


get player index
```python
index_list = []
index = sp.find_all(class_ = 'index-3cdMSKi7')
for i in range(0, len(index), 2):
    index_list.append(index[i].text)
```

In [4]:
# get player name
def get_player_name():
    player_list = []
    player = sp.find_all(class_ = 'full-3fV3c9pF')
    for i in range(0, len(player), 2):
        player_list.append(player[i].text + ' ' + player[i+1].text)
    return player_list

In [5]:
# get player's position
def get_player_position():
    position_list = []
    position = sp.find_all(class_ = 'position-28TbwVOg')
    for i in position:
        position_list.append(i.text)
    return position_list

In [6]:
def get_column_name():
    click_standard_button()
    sp = soup(driver.page_source, 'html.parser')
    
    data_column_name = ['PLAYER', 'POSITION', 'TEAM']
    temp = sp.find_all(class_ = 'bui-text cellheader bui-text')
    for i in range(4, len(temp), 2):
        data_column_name.append(temp[i].text)
    temp = sp.find_all(class_ = 'bui-text cellheader selected-1vxxHvFg bui-text')
    for i in range(0, len(temp), 2):
        data_column_name.append(temp[i].text)

    click_expanded_button()
    sp = soup(driver.page_source, 'html.parser')
    temp = sp.find_all(class_ = 'bui-text cellheader bui-text')
    for i in range(4, len(temp), 2):
        data_column_name.append(temp[i].text)
        
    return data_column_name

In [7]:
# click standard button
def click_standard_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[1]/button').click()
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[1]/button').click()
    time.sleep(3)
    sp = soup(driver.page_source, 'html.parser')

In [8]:
# click expanded button
def click_expanded_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[2]/button').click()
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[2]/button').click()
    time.sleep(3)

In [9]:
# click next page button
def click_next_page_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[3]/div[2]/div/div/div[2]/button').click()
    time.sleep(3)
    sp = soup(driver.page_source, 'html.parser')

In [10]:
# get standard data
def get_data():
    data = []
    
    click_standard_button()
    sp = soup(driver.page_source, 'html.parser')
    standard_data = []
    tt = sp.select('#stats-app-root tr td')
    for i in tt:
        standard_data.append(i.text)
    
    click_expanded_button()
    sp = soup(driver.page_source, 'html.parser')
    expanded_data = []
    tt = sp.select('#stats-app-root tr td')
    for i in tt:
        expanded_data.append(i.text)

    for i in range(len(standard_data)//17):
        data.extend(standard_data[i*17:i*17+17])
        data.extend(expanded_data[i*16+1:i*16+16])

    return data

```python
html = driver.get(url) 
# 有時候會 get 不完整，可以用這邊 reload 一次 XD
```

In [11]:
try:
    df_list = []
    while True:
        sp = soup(driver.page_source, 'html.parser')
        print('page start')
        time.sleep(2)
        name = get_player_name()
        time.sleep(2)
        position = get_player_position()
        time.sleep(2)
        current_data = get_data()
        time.sleep(2)
        for i in range(len(name)):
            row = []
            row.append(name[i])
            row.append(position[i])
            row.extend(current_data[i*32:i*32+32])
            df_list.append(row)
        click_next_page_button()
except:
    print('This is the last page :)')

page start
page start
This is the last page :)


In [12]:
df = pd.DataFrame(df_list, columns=get_column_name())
df

,PLAYER,POSITION,TEAM,G,AB,R,H,2B,3B,HR,...,GO/AO,XBH,TB,IBB,BABIP,ISO,AB/HR,BB/K,BB%,SO%
0,Seby Zavala,C,AZ,7,14,2,5,1,0,0,...,0.75,1,6,0,.417,.071,-.--,1.000,.118,.118
1,Corbin Carroll,LF,AZ,155,565,116,161,30,10,25,...,1.01,65,286,1,.325,.221,22.60,.456,.088,.194
2,Ketel Marte,2B,AZ,150,569,94,157,26,9,25,...,1.01,60,276,2,.300,.209,22.76,.651,.109,.168
3,Christian Walker,1B,AZ,157,582,86,150,36,2,33,...,0.77,71,289,3,.272,.239,17.64,.488,.094,.192
4,Dominic Fletcher,CF,AZ,28,93,10,28,5,1,2,...,1.14,8,41,1,.377,.140,46.50,.318,.069,.216
5,Lourdes Gurriel,LF,AZ,145,551,65,144,35,2,24,...,1.05,61,255,1,.282,.201,22.96,.320,.056,.174
6,Gabriel Moreno,C,AZ,111,341,33,97,19,1,7,...,1.67,27,139,0,.338,.123,48.71,.387,.076,.197
7,Tommy Pham,DH,AZ,50,195,26,47,12,2,6,...,1.81,20,81,0,.289,.174,32.50,.360,.083,.230
8,Evan Longoria,3B,AZ,74,211,25,47,9,0,11,...,0.88,20,89,0,.277,.199,19.18,.315,.097,.308
9,Geraldo Perdomo,SS,AZ,144,407,71,100,20,4,6,...,0.78,30,146,1,.295,.113,67.83,.744,.129,.174


In [ ]:
df.to_csv('mlb_stats_test.csv', index=False, encoding='utf-8')

In [ ]:
driver.quit()